At this stage, we will prepare the downloaded data for individual pair analysis

In [1609]:
import numpy as np
import pandas as pd
import pickle

## IMPORT DATA

#### SHOW SAVED PAIRS

In [1610]:
# get dataframe from the previous step
df = pd.read_pickle("pairs_to_download.pkl")
print(df)

x_list = df.index.tolist()
y_list = df.columns.tolist()

new_pairs_list = [] # list of pairs as tuples (A,B)

for i in range(len(x_list)):
    for j in range(len(y_list)):
        if df.iloc[i,j]>0:
            new_pairs_list.append((x_list[i], y_list[j]))

new_pairs_list

          CNP       DUK        ES       OGE       RUN        SO       VST  \
AEE       NaN  0.035623       NaN       NaN       NaN       NaN       NaN   
AEP  0.031525       NaN       NaN  0.045966       NaN       NaN  0.020333   
CMS       NaN  0.009237       NaN       NaN       NaN       NaN       NaN   
CNP       NaN       NaN       NaN       NaN       NaN       NaN  0.018811   
ED        NaN       NaN       NaN       NaN       NaN  0.049074  0.002472   
EIX       NaN       NaN  0.013553       NaN       NaN       NaN       NaN   
NEE       NaN       NaN       NaN       NaN  0.039231       NaN       NaN   
NRG       NaN       NaN       NaN       NaN       NaN       NaN  0.033844   
OGE       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
SO        NaN       NaN       NaN       NaN       NaN       NaN  0.033580   
VST       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

          WEC       XEL  
AEE       NaN       NaN  
AEP       NaN       NaN

[('AEE', 'DUK'),
 ('AEP', 'CNP'),
 ('AEP', 'OGE'),
 ('AEP', 'VST'),
 ('CMS', 'DUK'),
 ('CNP', 'VST'),
 ('ED', 'SO'),
 ('ED', 'VST'),
 ('EIX', 'ES'),
 ('NEE', 'RUN'),
 ('NRG', 'VST'),
 ('OGE', 'XEL'),
 ('SO', 'VST'),
 ('VST', 'WEC'),
 ('VST', 'XEL')]

#### SELECT TICKERS & RESOLUTION TO READ DATA

In [1611]:
# PICK FILENAME FORMAT
# "Y": "MA_1D.csv" for downloads with yfinance
# "T": "NYSE_MA, 1D.csv" for downloads from TradingView

file_format = "Y"

# SELECT TICKERS
ticker1 ="CNP"
ticker2 ="VST"

# need exchange info if fileformat is T
ticker1_exchange = 'NYSE'
ticker2_exchange = 'NYSE'

# SELECT RESOLUTION
# 5m,30m,1h,1d
resolution = '1D'

if file_format == "T":
    filename1 = "data/"+ ticker1_exchange.upper() +"_" + ticker1 + ", " + resolution.upper() + ".csv"
    filename2 = "data/"+ ticker2_exchange.upper() +"_" + ticker2 + ", " + resolution.upper() + ".csv"
elif file_format == "Y":
    filename1 = "data/"+ ticker1 +"_" + resolution.upper() + ".csv"
    filename2 = "data/"+ ticker2 +"_" + resolution.upper() + ".csv"
else:
    filename1 = "data/NYSE_MA, 1D.csv"
    filename2 = "data/NYSE_V, 1D.csv"


#### READ SAMPLE DATA

In [1612]:
# get a sample from file1
import random
p = 0.01  # 1% of all the lines
df1_sample = pd.read_csv(filename1,header=0, skiprows=lambda i: i>0 and random.random() > p)
df1_sample.head()

,time,Open,High,Low,Close,Adj Close,Volume,ticker
0,2022-03-09 00:00:00-05:00,28.799999,28.90,28.34,28.379999,28.071444,3795400,CNP
1,2022-09-16 00:00:00-04:00,32.080002,32.18,31.83,32.040001,32.040001,6396700,CNP


In [1613]:
# get a sample from file2
df2_sample = pd.read_csv(filename2,header=0, skiprows=lambda i: i>0 and random.random() > p)
df2_sample.head()

,time,Open,High,Low,Close,Adj Close,Volume,ticker
0,2022-01-05 00:00:00-05:00,22.48,22.760000,22.090000,22.110001,21.612705,4037900,VST
1,2022-09-14 00:00:00-04:00,24.98,25.370001,24.959999,25.360001,25.177868,3449800,VST


#### GET ALL DATA

In [1614]:
df1 = pd.read_csv(filename1, parse_dates=["time"])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       253 non-null    object 
 1   Open       253 non-null    float64
 2   High       253 non-null    float64
 3   Low        253 non-null    float64
 4   Close      253 non-null    float64
 5   Adj Close  253 non-null    float64
 6   Volume     253 non-null    int64  
 7   ticker     253 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 15.9+ KB


In [1615]:
df2=pd.read_csv(filename2, parse_dates=["time"])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       253 non-null    object 
 1   Open       253 non-null    float64
 2   High       253 non-null    float64
 3   Low        253 non-null    float64
 4   Close      253 non-null    float64
 5   Adj Close  253 non-null    float64
 6   Volume     253 non-null    int64  
 7   ticker     253 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 15.9+ KB


#### FILTER

In [1616]:
df1.columns = map(str.lower, df1.columns)
df1

,time,open,high,low,close,adj close,volume,ticker
0,2021-10-21 00:00:00-04:00,26.610001,26.719999,26.469999,26.510000,25.890596,3389700,CNP
1,2021-10-22 00:00:00-04:00,26.559999,26.670000,26.480000,26.540001,25.919897,5155500,CNP
2,2021-10-25 00:00:00-04:00,26.510000,26.860001,26.370001,26.770000,26.144522,3868000,CNP
3,2021-10-26 00:00:00-04:00,26.790001,26.930000,26.660000,26.680000,26.056627,2946000,CNP
4,2021-10-27 00:00:00-04:00,26.670000,26.690001,26.309999,26.309999,25.695269,3292000,CNP
...,...,...,...,...,...,...,...,...
248,2022-10-17 00:00:00-04:00,26.230000,26.650000,26.170000,26.250000,26.250000,4428300,CNP
249,2022-10-18 00:00:00-04:00,26.690001,27.000000,26.570000,26.959999,26.959999,6471100,CNP
250,2022-10-19 00:00:00-04:00,26.600000,27.120001,26.549999,27.000000,27.000000,6063700,CNP
251,2022-10-20 00:00:00-04:00,26.950001,27.040001,26.400000,26.600000,26.600000,6638100,CNP


In [1617]:
df1.columns = map(str.lower, df1.columns)
df1 = df1[['time','close','volume']]
df1 = df1.rename(columns={"time": "datetime"})
df1['datetime'] = pd.to_datetime(df1['datetime'], utc=True)  # be sure that you get datetime

# we don't want null or zero values (price & volume), use forward fill
df1['close'] = df1['close'].fillna(method='ffill')
df1['volume'] = df1['volume'].replace(to_replace=0, method='ffill')
df1

,datetime,close,volume
0,2021-10-21 04:00:00+00:00,26.510000,3389700
1,2021-10-22 04:00:00+00:00,26.540001,5155500
2,2021-10-25 04:00:00+00:00,26.770000,3868000
3,2021-10-26 04:00:00+00:00,26.680000,2946000
4,2021-10-27 04:00:00+00:00,26.309999,3292000
...,...,...,...
248,2022-10-17 04:00:00+00:00,26.250000,4428300
249,2022-10-18 04:00:00+00:00,26.959999,6471100
250,2022-10-19 04:00:00+00:00,27.000000,6063700
251,2022-10-20 04:00:00+00:00,26.600000,6638100


In [1618]:
df2.columns = map(str.lower, df2.columns)
df2 = df2[['time','close','volume']]
df2 = df2.rename(columns={"time": "datetime"})
df2['datetime'] = pd.to_datetime(df2['datetime'], utc=True)  # be sure that you get datetime

# we don't want null or zero values (price & volume), use forward fill
df2['close'] = df2['close'].fillna(method='ffill')
df2['volume'] = df2['volume'].replace(to_replace=0, method='ffill')
df2

,datetime,close,volume
0,2021-10-21 04:00:00+00:00,19.100000,2516600
1,2021-10-22 04:00:00+00:00,19.110001,2656300
2,2021-10-25 04:00:00+00:00,19.500000,6030600
3,2021-10-26 04:00:00+00:00,19.500000,3051200
4,2021-10-27 04:00:00+00:00,19.660000,3035100
...,...,...,...
248,2022-10-17 04:00:00+00:00,22.030001,3498300
249,2022-10-18 04:00:00+00:00,22.559999,2961700
250,2022-10-19 04:00:00+00:00,22.200001,3410100
251,2022-10-20 04:00:00+00:00,21.520000,4054200


#### ADD TICKER INFORMATION

In [1619]:
# if not added already in the previous steps

In [1620]:
df1_1 = df1.copy();
if not 'ticker' in df1_1.columns:
    df1_1['ticker'] = ticker1;
df1_1.head()

,datetime,close,volume,ticker
0,2021-10-21 04:00:00+00:00,26.510000,3389700,CNP
1,2021-10-22 04:00:00+00:00,26.540001,5155500,CNP
2,2021-10-25 04:00:00+00:00,26.770000,3868000,CNP
3,2021-10-26 04:00:00+00:00,26.680000,2946000,CNP
4,2021-10-27 04:00:00+00:00,26.309999,3292000,CNP


In [1621]:
df2_1 = df2.copy();
if not 'ticker' in df2_1.columns:
    df2_1['ticker'] = ticker2;
df2_1.head()

,datetime,close,volume,ticker
0,2021-10-21 04:00:00+00:00,19.100000,2516600,VST
1,2021-10-22 04:00:00+00:00,19.110001,2656300,VST
2,2021-10-25 04:00:00+00:00,19.500000,6030600,VST
3,2021-10-26 04:00:00+00:00,19.500000,3051200,VST
4,2021-10-27 04:00:00+00:00,19.660000,3035100,VST


## METHOD 1 (CONCATENATE  & KEEP ALL ROWS)

In [1622]:
df12_c = pd.concat([df2_1,df1_1]).reset_index(drop=True) # need to drop old index values
df12_c

,datetime,close,volume,ticker
0,2021-10-21 04:00:00+00:00,19.100000,2516600,VST
1,2021-10-22 04:00:00+00:00,19.110001,2656300,VST
2,2021-10-25 04:00:00+00:00,19.500000,6030600,VST
3,2021-10-26 04:00:00+00:00,19.500000,3051200,VST
4,2021-10-27 04:00:00+00:00,19.660000,3035100,VST
...,...,...,...,...
501,2022-10-17 04:00:00+00:00,26.250000,4428300,CNP
502,2022-10-18 04:00:00+00:00,26.959999,6471100,CNP
503,2022-10-19 04:00:00+00:00,27.000000,6063700,CNP
504,2022-10-20 04:00:00+00:00,26.600000,6638100,CNP


In [1624]:
#df12_c_agg = df12_c.groupby('DateTime').agg({'ticker':['close','Volume']})
df12_c_agg = df12_c.groupby('ticker')
df12_c_agg.head()

,datetime,close,volume,ticker
0,2021-10-21 04:00:00+00:00,19.100000,2516600,VST
1,2021-10-22 04:00:00+00:00,19.110001,2656300,VST
2,2021-10-25 04:00:00+00:00,19.500000,6030600,VST
3,2021-10-26 04:00:00+00:00,19.500000,3051200,VST
4,2021-10-27 04:00:00+00:00,19.660000,3035100,VST
253,2021-10-21 04:00:00+00:00,26.510000,3389700,CNP
254,2021-10-22 04:00:00+00:00,26.540001,5155500,CNP
255,2021-10-25 04:00:00+00:00,26.770000,3868000,CNP
256,2021-10-26 04:00:00+00:00,26.680000,2946000,CNP
257,2021-10-27 04:00:00+00:00,26.309999,3292000,CNP


In [1625]:
df12_c_1 = df12_c.set_index(["ticker", "datetime"]).sort_index() # set indexes
df12_c_1.head()

close   volume
ticker datetime                                     
CNP    2021-10-21 04:00:00+00:00  26.510000  3389700
       2021-10-22 04:00:00+00:00  26.540001  5155500
       2021-10-25 04:00:00+00:00  26.770000  3868000
       2021-10-26 04:00:00+00:00  26.680000  2946000
       2021-10-27 04:00:00+00:00  26.309999  3292000

#### SLICE DATAFRAME TO GET TICKER DATA

In [1626]:
df12_c_1.xs(ticker1)

,close,volume
datetime,,
2021-10-21 04:00:00+00:00,26.510000,3389700
2021-10-22 04:00:00+00:00,26.540001,5155500
2021-10-25 04:00:00+00:00,26.770000,3868000
2021-10-26 04:00:00+00:00,26.680000,2946000
2021-10-27 04:00:00+00:00,26.309999,3292000
...,...,...
2022-10-17 04:00:00+00:00,26.250000,4428300
2022-10-18 04:00:00+00:00,26.959999,6471100
2022-10-19 04:00:00+00:00,27.000000,6063700


In [1627]:
df12_c_1.xs(ticker2)

,close,volume
datetime,,
2021-10-21 04:00:00+00:00,19.100000,2516600
2021-10-22 04:00:00+00:00,19.110001,2656300
2021-10-25 04:00:00+00:00,19.500000,6030600
2021-10-26 04:00:00+00:00,19.500000,3051200
2021-10-27 04:00:00+00:00,19.660000,3035100
...,...,...
2022-10-17 04:00:00+00:00,22.030001,3498300
2022-10-18 04:00:00+00:00,22.559999,2961700
2022-10-19 04:00:00+00:00,22.200001,3410100


In [1628]:
# END OF METHOD 1, CONTINUE WITH METHOD 2

## METHOD 2 (MERGE TICKERS  & KEEP ONLY INNER TIME DATA)

In [1629]:
df12_m = pd.merge(df1_1, df2_1, on="datetime")
df12_m.head()

,datetime,close_x,volume_x,ticker_x,close_y,volume_y,ticker_y
0,2021-10-21 04:00:00+00:00,26.510000,3389700,CNP,19.100000,2516600,VST
1,2021-10-22 04:00:00+00:00,26.540001,5155500,CNP,19.110001,2656300,VST
2,2021-10-25 04:00:00+00:00,26.770000,3868000,CNP,19.500000,6030600,VST
3,2021-10-26 04:00:00+00:00,26.680000,2946000,CNP,19.500000,3051200,VST
4,2021-10-27 04:00:00+00:00,26.309999,3292000,CNP,19.660000,3035100,VST


#### TIME ZONE CONVERSION (IF NEEDED)

In [1630]:
df12_m['datetime'] = pd.to_datetime(df12_m['datetime'], utc=True)
#df12_m.datetime = df12_m.datetime.dt.tz_convert("Europe/Istanbul")
#df12_m.datetime = df12_m.datetime.dt.tz_convert("Canada/Pacific")
df12_m.datetime = df12_m.datetime.dt.tz_convert("UTC")
df12_m

,datetime,close_x,volume_x,ticker_x,close_y,volume_y,ticker_y
0,2021-10-21 04:00:00+00:00,26.510000,3389700,CNP,19.100000,2516600,VST
1,2021-10-22 04:00:00+00:00,26.540001,5155500,CNP,19.110001,2656300,VST
2,2021-10-25 04:00:00+00:00,26.770000,3868000,CNP,19.500000,6030600,VST
3,2021-10-26 04:00:00+00:00,26.680000,2946000,CNP,19.500000,3051200,VST
4,2021-10-27 04:00:00+00:00,26.309999,3292000,CNP,19.660000,3035100,VST
...,...,...,...,...,...,...,...
248,2022-10-17 04:00:00+00:00,26.250000,4428300,CNP,22.030001,3498300,VST
249,2022-10-18 04:00:00+00:00,26.959999,6471100,CNP,22.559999,2961700,VST
250,2022-10-19 04:00:00+00:00,27.000000,6063700,CNP,22.200001,3410100,VST
251,2022-10-20 04:00:00+00:00,26.600000,6638100,CNP,21.520000,4054200,VST


#### GET START & END DATES & SET INDEX

In [1631]:
# get start and end days of time series
min_date, max_date = df12_m.datetime.min(), df12_m.datetime.max()
#str(min_date), str(max_date)
min_date.strftime('%m/%d/%Y'), max_date.strftime('%m/%d/%Y')

('10/21/2021', '10/21/2022')

In [1632]:
df12_m = df12_m.set_index("datetime").sort_index()
df12_m.head()

,close_x,volume_x,ticker_x,close_y,volume_y,ticker_y
datetime,,,,,,
2021-10-21 04:00:00+00:00,26.510000,3389700,CNP,19.100000,2516600,VST
2021-10-22 04:00:00+00:00,26.540001,5155500,CNP,19.110001,2656300,VST
2021-10-25 04:00:00+00:00,26.770000,3868000,CNP,19.500000,6030600,VST
2021-10-26 04:00:00+00:00,26.680000,2946000,CNP,19.500000,3051200,VST
2021-10-27 04:00:00+00:00,26.309999,3292000,CNP,19.660000,3035100,VST


#### GET CUSTOM DATES (IF NEEDED):

In [1633]:
# CUSTOM DATE
import datetime
from_date = datetime.datetime(2018, 6, 1)
to_date = datetime.datetime(2022, 1, 1)
# from_date = '2021-01-01 13:30:00+00:00' # UTC
# to_date = '2022-01-01 13:30:00+00:00' # UTC
# date_format = '%Y-%m-%d'
# from_date = datetime.strptime(from_date, date_format)
# to_date = datetime.strptime(to_date, date_format)

new_index = pd.date_range(from_date, to_date, freq='1D', tz='UTC')
new_index

DatetimeIndex(['2018-06-01 00:00:00+00:00', '2018-06-02 00:00:00+00:00',
               '2018-06-03 00:00:00+00:00', '2018-06-04 00:00:00+00:00',
               '2018-06-05 00:00:00+00:00', '2018-06-06 00:00:00+00:00',
               '2018-06-07 00:00:00+00:00', '2018-06-08 00:00:00+00:00',
               '2018-06-09 00:00:00+00:00', '2018-06-10 00:00:00+00:00',
               ...
               '2021-12-23 00:00:00+00:00', '2021-12-24 00:00:00+00:00',
               '2021-12-25 00:00:00+00:00', '2021-12-26 00:00:00+00:00',
               '2021-12-27 00:00:00+00:00', '2021-12-28 00:00:00+00:00',
               '2021-12-29 00:00:00+00:00', '2021-12-30 00:00:00+00:00',
               '2021-12-31 00:00:00+00:00', '2022-01-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1311, freq='D')

In [1634]:
df12_m_custom = df12_m.reindex(new_index, method="ffill")
df12_m_custom

,close_x,volume_x,ticker_x,close_y,volume_y,ticker_y
2018-06-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-12-28 00:00:00+00:00,27.580000,2132800.0,CNP,22.190001,2283600.0,VST
2021-12-29 00:00:00+00:00,27.850000,2119100.0,CNP,22.049999,1864300.0,VST
2021-12-30 00:00:00+00:00,27.950001,2618600.0,CNP,21.870001,2895400.0,VST
2021-12-31 00:00:00+00:00,27.830000,2815300.0,CNP,22.389999,7636100.0,VST


#### SAVE TO FILE FOR MORE ANALYSIS

In [1635]:
df12_m.to_pickle("saved.pkl")
if 'df12_m_custom' in locals():
    df12_m_custom.to_pickle("saved_custom.pkl")